In [5]:
import pandas as pd
import pdb
import numpy as np
import itertools
from snakemake.io import expand
import yaml
import os
import sys
import pathlib
import pyranges as pr
import cerberus 

p = os.path.dirname(os.getcwd())
sys.path.append(p)

from utils import *
from sm_utils import *
from humanized_utils import *

In [6]:
config_file = 'config.yml'
with open(config_file) as f:
    config = yaml.safe_load(f)

## 231108 merge pseudochrom gtfs

In [9]:
# config_tsv = '231027_config.tsv'
# config_tsv = 'test_pseudochrom_parsing_config.tsv'
# config_tsv = '231102_config_ad003.tsv'
config_tsv = '231106_config_ad008_ad003.tsv'
p_meta_tsv = 'pseudochromosome_metadata.tsv'
meta_tsv = 'mouse_metadata.tsv'
auto_dedupe = True

df, p_df = parse_config_file(config_tsv,
                       meta_tsv,
                       p_meta_tsv,
                       auto_dedupe=auto_dedupe)

In [11]:
p_df.genotype.unique()

array(['5xCLU-h2kbKI_HO', 'CLU-h2kbKI_HO', '5xFADHEMI', '5xFADWT',
       'hABKI_Swe_IB_HET', 'hABKI_Swe_IB_WT'], dtype=object)

In [13]:
wc = {'genotype': '5xCLU-h2kbKI_HO'}
get_df_col(wc,
                    p_df,
                    config['ref']['pseudochrom']['human_gene']['fmt_gtf'])

['human_ref/pseudochrom/hCLU/CLU/CLU_fmt.gtf']

In [7]:
mouse_gtf = 'ref/pseudochrom/hCLU/Clu/Clu_fmt.gtf'
human_gtf = 'human_ref/pseudochrom/hCLU/CLU/CLU_fmt.gtf'
pseudochrom = 'hCLU'
wc = {'pseudochrom': 'hCLU'}
ofile = 'test_sorted_gtf.gtf'

In [8]:
merge_sort_human_mouse_pseudochrom_gtfs(wc,
                                            mouse_gtf,
                                            human_gtf,
                                            ofile)

In [29]:
m_df = pr.read_gtf(mouse_gtf).df
h_df = pr.read_gtf(human_gtf).df

In [30]:
m_df.head()

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_name,gene_status,source_attr,transcript_id,transcript_status,transcript_name,exon_number,exon_id,exon_status
0,hCLU,TALON,gene,3000,10510,.,+,.,hCLU,hCLU,KNOWN,TALON,NaN,NaN,NaN,NaN,NaN,NaN
1,hCLU,TALON,transcript,3000,15595,.,+,.,hCLU,hCLU,KNOWN,NaN,mouse_Clu_TALONT000141863,KNOWN,mouse_Clu_TALONT000141863,NaN,NaN,NaN
2,hCLU,TALON,exon,3000,3206,.,+,.,hCLU,hCLU,KNOWN,NaN,mouse_Clu_TALONT000141863,KNOWN,mouse_Clu_TALONT000141863,1,730959,KNOWN
3,hCLU,TALON,exon,6249,6370,.,+,.,hCLU,hCLU,KNOWN,NaN,mouse_Clu_TALONT000141863,KNOWN,mouse_Clu_TALONT000141863,2,730947,KNOWN
4,hCLU,TALON,exon,7842,7991,.,+,.,hCLU,hCLU,KNOWN,NaN,mouse_Clu_TALONT000141863,KNOWN,mouse_Clu_TALONT000141863,3,730949,KNOWN


In [31]:
h_df.head()

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_name,gene_status,source_attr,transcript_id,transcript_status,transcript_name,exon_number,exon_id,exon_status
0,hCLU,TALON,gene,2970,16063,.,+,.,hCLU,hCLU,KNOWN,TALON,NaN,NaN,NaN,NaN,NaN,NaN
1,hCLU,TALON,transcript,2970,17142,.,+,.,hCLU,hCLU,KNOWN,NaN,human_CLU_TALONT000141863,KNOWN,human_CLU_TALONT000141863,NaN,NaN,NaN
2,hCLU,TALON,exon,2970,3206,.,+,.,hCLU,hCLU,KNOWN,NaN,human_CLU_TALONT000141863,KNOWN,human_CLU_TALONT000141863,1,730961,KNOWN
3,hCLU,TALON,exon,6249,6370,.,+,.,hCLU,hCLU,KNOWN,NaN,human_CLU_TALONT000141863,KNOWN,human_CLU_TALONT000141863,2,730947,KNOWN
4,hCLU,TALON,exon,7842,7991,.,+,.,hCLU,hCLU,KNOWN,NaN,human_CLU_TALONT000141863,KNOWN,human_CLU_TALONT000141863,3,730949,KNOWN


In [32]:
# concat and update the 
df = pd.concat([m_df, h_df], ignore_index=True, axis=0)
gene_entries = df.loc[df.Feature=='gene'].copy(deep=True)
if len(gene_entries.index) == 2:
    gene_end = gene_entries.End.max(0)
    gene_start = gene_entries.Start.min(0)
    df.drop(gene_entries.index.tolist()[0], axis=0, inplace=True)
    df.loc[df.Feature=='gene', 'Start'] = gene_start
    df.loc[df.Feature=='gene', 'End'] = gene_end

In [33]:
df.loc[df.Feature=='gene']

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_name,gene_status,source_attr,transcript_id,transcript_status,transcript_name,exon_number,exon_id,exon_status
51,hCLU,TALON,gene,2970,16063,.,+,.,hCLU,hCLU,KNOWN,TALON,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
df['gene_id'] = pseudochrom
df['gene_name'] = pseudochrom

In [35]:
df.gene_name.unique()

array(['hCLU'], dtype=object)

In [36]:
df.head()

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_name,gene_status,source_attr,transcript_id,transcript_status,transcript_name,exon_number,exon_id,exon_status
1,hCLU,TALON,transcript,3000,15595,.,+,.,hCLU,hCLU,KNOWN,NaN,mouse_Clu_TALONT000141863,KNOWN,mouse_Clu_TALONT000141863,NaN,NaN,NaN
2,hCLU,TALON,exon,3000,3206,.,+,.,hCLU,hCLU,KNOWN,NaN,mouse_Clu_TALONT000141863,KNOWN,mouse_Clu_TALONT000141863,1,730959,KNOWN
3,hCLU,TALON,exon,6249,6370,.,+,.,hCLU,hCLU,KNOWN,NaN,mouse_Clu_TALONT000141863,KNOWN,mouse_Clu_TALONT000141863,2,730947,KNOWN
4,hCLU,TALON,exon,7842,7991,.,+,.,hCLU,hCLU,KNOWN,NaN,mouse_Clu_TALONT000141863,KNOWN,mouse_Clu_TALONT000141863,3,730949,KNOWN
5,hCLU,TALON,exon,9360,9531,.,+,.,hCLU,hCLU,KNOWN,NaN,mouse_Clu_TALONT000141863,KNOWN,mouse_Clu_TALONT000141863,4,730951,KNOWN


In [37]:
# try again w/ correct gtfs
df_sorted = cerberus.sort_gtf(df)

In [38]:
df_sorted.head()

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_name,gene_status,source_attr,transcript_id,transcript_status,transcript_name,exon_number,exon_id,exon_status
0,hCLU,TALON,gene,2970,16063,.,+,.,hCLU,hCLU,KNOWN,TALON,NaN,NaN,NaN,NaN,NaN,NaN
1,hCLU,TALON,transcript,2970,17142,.,+,.,hCLU,hCLU,KNOWN,NaN,human_CLU_TALONT000141863,KNOWN,human_CLU_TALONT000141863,NaN,NaN,NaN
2,hCLU,TALON,exon,2970,3206,.,+,.,hCLU,hCLU,KNOWN,NaN,human_CLU_TALONT000141863,KNOWN,human_CLU_TALONT000141863,1,730961,KNOWN
3,hCLU,TALON,exon,6249,6370,.,+,.,hCLU,hCLU,KNOWN,NaN,human_CLU_TALONT000141863,KNOWN,human_CLU_TALONT000141863,2,730947,KNOWN
4,hCLU,TALON,exon,7842,7991,.,+,.,hCLU,hCLU,KNOWN,NaN,human_CLU_TALONT000141863,KNOWN,human_CLU_TALONT000141863,3,730949,KNOWN


In [41]:
for t in df.transcript_id.unique():
    print(t)
df.loc[df.transcript_id.isnull()]

mouse_Clu_TALONT000141863
mouse_Clu_TALONT000141864
mouse_Clu_TALONT000141865
mouse_Clu_TALONT000141866
mouse_Clu_TALONT000141867
mouse_Clu_TALONT000141868
mouse_Clu_TALONT000141869
mouse_Clu_TALONT000141870
mouse_Clu_TALONT000141871
nan
human_CLU_TALONT000141863
human_CLU_TALONT000141864
human_CLU_TALONT000141865
human_CLU_TALONT000141866
human_CLU_TALONT000141867
human_CLU_TALONT000141868
human_CLU_TALONT000141869
human_CLU_TALONT000141870
human_CLU_TALONT000141871
human_CLU_TALONT000141872


,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_name,gene_status,source_attr,transcript_id,transcript_status,transcript_name,exon_number,exon_id,exon_status,feature_rank
51,hCLU,TALON,gene,2970,16063,.,+,.,hCLU,hCLU,KNOWN,TALON,NaN,NaN,NaN,NaN,NaN,NaN,0


## 231107 try to make cerberus work again...

In [10]:
config_tsv = '231027_config.tsv'
# config_tsv = 'test_pseudochrom_parsing_config.tsv'
# config_tsv = '231102_config_ad003.tsv'
p_meta_tsv = 'pseudochromosome_metadata.tsv'
meta_tsv = 'mouse_metadata.tsv'
auto_dedupe = True

df, p_df = parse_config_file(config_tsv,
                       meta_tsv,
                       p_meta_tsv,
                       auto_dedupe=auto_dedupe)

/Users/fairliereese/Documents/programming/mortazavi_lab/bin/modelad_pipeline/proc_update/sm_utils.py:128: UserWarning: Config found non-pseudochrom mouse w/ genotypes ['hABKI_SWE'], is this expected?
  warnings.warn(f'Config found non-pseudochrom mouse w/ genotypes {genotypes}, is this expected?')


In [12]:
p_df.cerberus_run.unique()

array(['1', '2', '4', '5', '15', '16', '17', '18', '19', '20', '21', '3',
       '6', '7', '8', '9', '10', '11', '12', '13', '14', '22', '23', '24',
       '25'], dtype=object)

In [13]:
max_cerb = df.loc[df.cerberus_run.astype(int)==df.cerberus_run.astype(int).max(axis=0)]


In [14]:
expand(expand(config['cerberus']['agg']['ends'],
               zip,
               study=max_cerb.study.tolist(),
               genotype=max_cerb.genotype.tolist(),
               sex=max_cerb.sex.tolist(),
               age=max_cerb.age.tolist(),
               tissue=max_cerb.tissue.tolist(),
               cerberus_run=max_cerb.cerberus_run.tolist(),
               allow_missing=True),
               end_mode='tss')

['data/cerberus/agg/ad008_hABKI_Swe_IB_WT_M_8_weeks_HC_25_tss.bed',
 'data/cerberus/agg/ad008_hABKI_Swe_IB_WT_M_8_weeks_HC_25_tss.bed',
 'data/cerberus/agg/ad008_hABKI_Swe_IB_WT_M_8_weeks_HC_25_tss.bed']

In [4]:
wc = {'cerberus_run': 2, 'end_mode': 'tss'}
get_prev_cerb_entry(wc, p_df, config['cerberus']['agg']['ends'], config)
                    

'data/cerberus/agg/ad002_5xFADHEMI_M_18_months_CTX_1_tss.bed'

In [7]:
wc = {'cerberus_run': 2}
get_prev_cerb_entry(wc, p_df, config['cerberus']['agg']['ics'], config)
                    

'data/cerberus/agg/ad002_5xFADHEMI_M_18_months_CTX_1_ic.tsv'

In [5]:
wc = {'cerberus_run': 1, 'end_mode': 'tss'}
get_prev_cerb_entry(wc, p_df, config['cerberus']['agg']['ends'], config)
                    

'ref/ca_tss.bed'

In [6]:
wc = {'cerberus_run': 1}
get_prev_cerb_entry(wc, p_df, config['cerberus']['agg']['ics'], config)
                    

'ref/ca_ic.tsv'

In [9]:
max_cerb = df.loc[df.cerberus_run==df.cerberus_run.max(axis=0)]
max_cerb

,batch,platform,fname,basename,path,flowcell,file_stem,chop_num,sample_temp,mouse_id,...,tissue,pseudochrom_needed,pseudochrom,Unnamed: 7,Unnamed: 8,sample,biorep_num,dataset,source,cerberus_run
34,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad004_18876_lig-blk_1_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad004_18876_lig-blk_1,1.0,ad004_18876,18876,...,HC,True,"(hTREM2,)",NaN,NaN,hTREM2-R47HKI_HET_M_10_weeks_HC,1,hTREM2-R47HKI_HET_M_10_weeks_HC_1,ad004_hTREM2-R47HKI_HET_M_10_weeks_HC,9
35,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad004_18876_lig-blk_2_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad004_18876_lig-blk_2,1.0,ad004_18876,18876,...,HC,True,"(hTREM2,)",NaN,NaN,hTREM2-R47HKI_HET_M_10_weeks_HC,1,hTREM2-R47HKI_HET_M_10_weeks_HC_1,ad004_hTREM2-R47HKI_HET_M_10_weeks_HC,9


In [15]:
ics = expand(config['cerberus']['ics'], 
    zip,
    study=df.study.tolist(),
    genotype=df.genotype.tolist(),
    sex=df.sex.tolist(),
    age=df.age.tolist(),
    tissue=df.tissue.tolist(),
    cerberus_run=df.cerberus_run.tolist()) 
tsss = expand(expand(config['cerberus']['ends'], 
    zip,
    study=df.study.tolist(),
    genotype=df.genotype.tolist(),
    sex=df.sex.tolist(),
    age=df.age.tolist(),
    tissue=df.tissue.tolist(),
    cerberus_run=df.cerberus_run.tolist(),
    allow_missing=True),
    end_mode='tss') 
tess = expand(expand(config['cerberus']['ends'], 
    zip,
    study=df.study.tolist(),
    genotype=df.genotype.tolist(),
    sex=df.sex.tolist(),
    age=df.age.tolist(),
    tissue=df.tissue.tolist(),
    cerberus_run=df.cerberus_run.tolist(),
    allow_missing=True),
    end_mode='tes') 
                                

In [16]:
for f in ics+tsss+tess:
    os.makedirs(os.path.dirname(f), exist_ok=True)
    pathlib.Path(f).touch()

In [17]:
df.genotype.unique().tolist()

['5xFADHEMI',
 '5xFADWT',
 '5xCLU-h2kbKI_HO',
 'CLU-h2kbKI_HO',
 'hTREM2-R47HKI_HET',
 'hTREM2_KI_HET',
 'hTREM2_KI_WT',
 '5xBIN1_HO',
 'BIN1_HO',
 'C57B6J',
 'hABKI_SWE',
 'ABCA7_HO',
 'hABKI_Swe_IB_HET',
 'hABKI_Swe_IB_WT']

## 231103 request all output

In [36]:
config['ref']['pseudochrom']['fa']

'ref/pseudochrom/{pseudochrom}/ref.fa'

In [33]:
# config_tsv = '231020_config.tsv'
# config_tsv = 'test_pseudochrom_parsing_config.tsv'
config_tsv = '231102_config_ad003.tsv'
p_meta_tsv = 'pseudochromosome_metadata.tsv'
meta_tsv = 'mouse_metadata.tsv'
auto_dedupe = True

df, p_df = parse_config_file(config_tsv,
                       meta_tsv,
                       p_meta_tsv,
                       auto_dedupe=auto_dedupe)

In [29]:
p_df[['pseudochrom', 'mouse_gene', 'human_gene']].drop_duplicates()

,pseudochrom,mouse_gene,human_gene
0,hCLU,Clu,CLU
12,dummy,dummy,dummy


In [30]:
list(set(expand(config['ref']['pseudochrom']['gene']['config'],
       zip,
       pseudochrom=p_df.pseudochrom.tolist(),
       mouse_gene=p_df.mouse_gene.tolist())))
list(set(expand(config['ref']['pseudochrom']['human_gene']['config'],
       zip,
       pseudochrom=p_df.pseudochrom.tolist(),
       human_gene=p_df.human_gene.tolist())))

['ref/pseudochrom/dummy/dummy/config.csv',
 'ref/pseudochrom/hCLU/Clu/config.csv']

In [32]:
list(set(expand(config['ref']['pseudochrom']['human_gene']['config'],
       zip,
       pseudochrom=p_df.pseudochrom.tolist(),
       human_gene=p_df.human_gene.tolist())))

['human_ref/pseudochrom/hCLU/CLU/config.csv',
 'human_ref/pseudochrom/dummy/dummy/config.csv']

In [ ]:
ref/pseudochrom/{pseudochrom}/{mouse_gene}/config.csv

In [19]:
p_df.head()

,platform,fname,basename,path,flowcell,file_stem,chop_num,sample_temp,mouse_id,study,...,Unnamed: 8,sample,biorep_num,dataset,source,cerberus_run,human_gene,mouse_gene,locus_type,notes
0,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_12517_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_12517_lig-blk_1,1.0,ad003_12517,12517,ad003,...,NaN,5xCLU-h2kbKI_HO_F_4_months_HC,1,5xCLU-h2kbKI_HO_F_4_months_HC_1,ad003_5xCLU-h2kbKI_HO_F_4_months_HC,1,CLU,Clu,chimeric,Some full exons are human; some are mouse
1,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_12517_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_12517_lig-blk_2,1.0,ad003_12517,12517,ad003,...,NaN,5xCLU-h2kbKI_HO_F_4_months_HC,1,5xCLU-h2kbKI_HO_F_4_months_HC_1,ad003_5xCLU-h2kbKI_HO_F_4_months_HC,1,CLU,Clu,chimeric,Some full exons are human; some are mouse
2,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_12649_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_12649_lig-blk_1,1.0,ad003_12649,12649,ad003,...,NaN,5xCLU-h2kbKI_HO_F_4_months_HC,2,5xCLU-h2kbKI_HO_F_4_months_HC_2,ad003_5xCLU-h2kbKI_HO_F_4_months_HC,1,CLU,Clu,chimeric,Some full exons are human; some are mouse
3,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_12649_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_12649_lig-blk_2,1.0,ad003_12649,12649,ad003,...,NaN,5xCLU-h2kbKI_HO_F_4_months_HC,2,5xCLU-h2kbKI_HO_F_4_months_HC_2,ad003_5xCLU-h2kbKI_HO_F_4_months_HC,1,CLU,Clu,chimeric,Some full exons are human; some are mouse
4,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_12659_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_12659_lig-blk_1,1.0,ad003_12659,12659,ad003,...,NaN,5xCLU-h2kbKI_HO_F_4_months_HC,3,5xCLU-h2kbKI_HO_F_4_months_HC_3,ad003_5xCLU-h2kbKI_HO_F_4_months_HC,1,CLU,Clu,chimeric,Some full exons are human; some are mouse


## 231103 get output files which are the mapped human / mouse genes to the pseudochromosome

In [3]:
# config_tsv = '231020_config.tsv'
# config_tsv = 'test_pseudochrom_parsing_config.tsv'
config_tsv = '231102_config_ad003.tsv'
p_meta_tsv = 'pseudochromosome_metadata.tsv'
meta_tsv = 'mouse_metadata.tsv'
auto_dedupe = True

df, p_df = parse_config_file(config_tsv,
                       meta_tsv,
                       p_meta_tsv,
                       auto_dedupe=auto_dedupe)

In [8]:
db = config['ref']['talon']['db']

In [16]:
def mk_pseudochrom_mapped_gene_talon_config(wc,
                                            species,
                                            p_df,
                                            ofile):    
    if species == 'mouse':
        cfg_entry = config['ref']['pseudochrom']['gene']['sort_bam']
        col = 'mouse_gene'
    elif species == 'human':
        cfg_entry = config['ref']['pseudochrom']['human_gene']['sort_bam']
        col = 'human_gene'
    gene = get_df_col(wc, p_df, col)
    
    if gene == 'dummy':
        pathlib.Path(ofile).touch()

    else:
        # input files
        bam = get_cfg_entries(wc, 
                              p_df,
                              cfg_entry)
        print(bam)

        # actual config file makery
        temp = get_cfg_entries(wildcards,
                                   p_df,
                                   cfg_entry,
                                   return_df=True)
        print(gene)
        print(temp)
        temp = temp[['sample', 'dataset', 'platform', 'file']]
        # temp_str = f'{species}_{gene}'
        # temp['sample'] = temp_str
        # temp['dataset'] = temp_str
        temp['sample'] = gene
        temp['dataset'] = gene
        print(temp)
        temp.to_csv(ofile, index=False, header=None, sep=',')

In [14]:
wildcards = {'genotype': '5xCLU-h2kbKI_HO', 'mouse_gene': 'Clu'}
species = 'mouse'

mk_pseudochrom_mapped_gene_talon_config(wildcards,
                                        species,
                                        p_df,
                                        'test_talon_cfg')

['ref/pseudochrom/hCLU/Clu/mapped_sorted.bam']
Clu
  platform                                              fname  \
0      ONT  /share/crsp/lab/seyedam/share/ad_nanopore_tmp/...   

                         basename  \
0  ad003_12517_lig-blk_1_t1.fastq   

                                                path flowcell  \
0  /share/crsp/lab/seyedam/share/ad_nanopore_tmp/...        1   

               file_stem  chop_num  sample_temp  mouse_id  study  ...  \
0  ad003_12517_lig-blk_1       1.0  ad003_12517     12517  ad003  ...   

                          sample biorep_num                          dataset  \
0  5xCLU-h2kbKI_HO_F_4_months_HC          1  5xCLU-h2kbKI_HO_F_4_months_HC_1   

                                source  cerberus_run human_gene  mouse_gene  \
0  ad003_5xCLU-h2kbKI_HO_F_4_months_HC             1        CLU         Clu   

   locus_type                                      notes  \
0    chimeric  Some full exons are human; some are mouse   

                        

In [7]:
wildcards = {'genotype': '5xFADWT', 'mouse_gene': 'dummy'}
species = 'mouse'

mk_pseudochrom_mapped_gene_talon_config(wildcards,
                                        species,
                                        p_df,
                                        'test_talon_cfg')

['ref/pseudochrom/dummy/dummy/mapped_sorted.bam']
dummy
   platform                                              fname  \
18      ONT  /share/crsp/lab/seyedam/share/ad_nanopore_tmp/...   

                          basename  \
18  ad003_11627_lig-blk_1_t1.fastq   

                                                 path flowcell  \
18  /share/crsp/lab/seyedam/share/ad_nanopore_tmp/...        1   

                file_stem  chop_num  sample_temp  mouse_id  study  ...  \
18  ad003_11627_lig-blk_1       1.0  ad003_11627     11627  ad003  ...   

                   sample biorep_num                  dataset  \
18  5xFADWT_F_4_months_HC          1  5xFADWT_F_4_months_HC_1   

                         source  cerberus_run human_gene  mouse_gene  \
18  ad003_5xFADWT_F_4_months_HC             3      dummy       dummy   

    locus_type  notes                                           file  
18       dummy  dummy  ref/pseudochrom/dummy/dummy/mapped_sorted.bam  

[1 rows x 28 columns]
   sample d

In [17]:
# for a given genotype, get the human and mouse gene mappings to 
# be given to talon
wildcards = {'genotype': '5xCLU-h2kbKI_HO'}
species = 'mouse'

if species == 'mouse':
    cfg_entry = config['ref']['pseudochrom']['gene']['sort_bam']
    col = 'mouse_gene'
elif species == 'human':
    cfg_entry = config['ref']['pseudochrom']['human_gene']['sort_bam']
    col = 'human_gene'
    
# input files
bam = get_cfg_entries(wc, 
                     p_df,
                     cfg_entry)
db = config['ref']['talon']['db']

# actual config file makery
gene = get_df_col(wc, p_df, col)
print(gene)
temp = get_cfg_entries(wildcards,
                       p_df,
                       cfg_entry,
                       return_df=True)
# print(bam)
# print(gene)
# print(temp)
temp = temp[['sample', 'dataset', 'platform', 'file']]
# temp_str = f'{species}_{gene}'
# temp['sample'] = temp_str
# temp['dataset'] = temp_str
temp['sample'] = gene
temp['dataset'] = gene
temp.to_csv(output.cfg, index=False, header=None, sep=',')



AttributeError: 'DataFrame' object has no attribute 'tolist'

In [29]:
# for a given genotype, get the human and mouse gene mappings to 
# be given to talon
wildcards = {'genotype': '5xCLU-h2kbKI_HO'}

# input files
mouse_bam = get_cfg_entries(wc, 
                     p_df,
                     config['ref']['gene']['sort_bam'])
human_bam = get_cfg_entries(wc, 
                     p_df,
                     config['human_ref']['gene']['sort_bam'])
db = config['ref']['talon']['db']

# actual config file makery
mouse_gene = get_df_col(wc, 
                 p_df, 
                 'mouse_gene')
human_gene = get_df_col(wc, p_df, 'human_gene')


print(mouse_bam)
print(mouse_gene)

print(human_bam)
print(human_gene)

# if both entries are dummy, just 

# if human_gene != 'dummy':
#     temp = get_cfg_entries(wildcards,
#                            p_df,
#                            config['human_ref']['gene']['sort_bam'],
#                            return_df=True)
# else: 
#     temp = pd.DataFrame()

# if mouse_gene != 'dummy':
    

['ref/pseudochrom/hCLU/Clu/mapped_sorted.bam']
Clu
['human_ref/pseudochrom/hCLU/CLU/mapped_sorted.bam']
CLU


In [31]:
db

'ref/talon/ref.db'

In [12]:


# wc = {'pseudochrom': 'hCLU'}

# input files


wc = {'genotype': '5xFADWT'}
eep = get_df_col(wc, 
                 p_df, 
                 'mouse_gene')
print(eep)

eep = get_cfg_entries(wc, 
                     p_df,
                     config['ref']['pseudochrom']['gene']['sort_bam'])
print(eep)
eep = get_cfg_entries(wc, 
                     p_df,
                     config['ref']['pseudochrom']['human_gene']['sort_bam'])
print(eep)

AttributeError: 'DataFrame' object has no attribute 'tolist'

In [14]:
p_df.genotype.tolist()

['5xCLU-h2kbKI_HO',
 '5xCLU-h2kbKI_HO',
 '5xCLU-h2kbKI_HO',
 '5xCLU-h2kbKI_HO',
 '5xCLU-h2kbKI_HO',
 '5xCLU-h2kbKI_HO',
 'CLU-h2kbKI_HO',
 'CLU-h2kbKI_HO',
 'CLU-h2kbKI_HO',
 'CLU-h2kbKI_HO',
 'CLU-h2kbKI_HO',
 'CLU-h2kbKI_HO',
 '5xFADHEMI',
 '5xFADHEMI',
 '5xFADHEMI',
 '5xFADHEMI',
 '5xFADHEMI',
 '5xFADHEMI',
 '5xFADWT',
 '5xFADWT',
 '5xFADWT',
 '5xFADWT',
 '5xFADWT',
 '5xFADWT']

In [ ]:
expand(config['ref']['gene']['ind_bam'],
       pseudochrom=

In [6]:
p_df[['pseudochrom', 'human_gene', 'mouse_gene']].drop_duplicates()

,pseudochrom,human_gene,mouse_gene
0,hCLU,CLU,Clu
12,dummy,dummy,dummy


In [12]:
wc = {'pseudochrom': 'hCLU'}
eep = get_cfg_entries(wc, 
                     p_df,
                     config['ref']['gene']['sort_bam'])
eep

['ref/pseudochrom/hCLU/Clu/mapped_sorted.bam']

In [13]:
wc = {'pseudochrom': 'hCLU'}
eep = get_cfg_entries(wc, 
                     p_df,
                     config['human_ref']['gene']['sort_bam'])
eep

['human_ref/pseudochrom/hCLU/CLU/mapped_sorted.bam']

AttributeError: 'DataFrame' object has no attribute 'human_gene'

## 231102 get input files for each genotype's reference

In [3]:
# config_tsv = '231020_config.tsv'
# config_tsv = 'test_pseudochrom_parsing_config.tsv'
config_tsv = '231102_config_ad003.tsv'
p_meta_tsv = 'pseudochromosome_metadata.tsv'
meta_tsv = 'mouse_metadata.tsv'
auto_dedupe = True

df, p_df = parse_config_file(config_tsv,
                       meta_tsv,
                       p_meta_tsv,
                       auto_dedupe=auto_dedupe)

In [4]:
wc = {'genotype': '5xCLU-h2kbKI_HO'}
get_df_col(wc, 
           p_df, 
           'pseudochrom',
           allow_multiple=True)

['hCLU']

In [5]:
['dummy'] == get_df_col(wc, 
           p_df, 
           'pseudochrom',
            allow_multiple=True)

False

In [6]:
df.genotype.unique()

array(['5xCLU-h2kbKI_HO', '5xFADHEMI', '5xFADWT', 'CLU-h2kbKI_HO'],
      dtype=object)

In [7]:
wc = {'genotype': '5xFADHEMI'}
get_df_col(wc, 
           p_df, 
           'pseudochrom',
           allow_multiple=True)

['dummy']

In [8]:
['dummy'] == get_df_col(wc, 
           p_df, 
           'pseudochrom',
            allow_multiple=True)

True

In [23]:
# config_tsv = '231020_config.tsv'
config_tsv = 'test_pseudochrom_parsing_config.tsv'
# config_tsv = '231102_config_ad003.tsv'
p_meta_tsv = 'pseudochromosome_metadata.tsv'
meta_tsv = 'mouse_metadata.tsv'
auto_dedupe = True

df, p_df = parse_config_file(config_tsv,
                       meta_tsv,
                       p_meta_tsv,
                       auto_dedupe=auto_dedupe)

In [24]:
wc = {'genotype': ('hgenotype1_genotype2')}
get_df_col(wc, 
           p_df, 
           'pseudochrom',
           allow_multiple=True)

['hgenotype1', 'hgenotype2']

In [25]:
['dummy'] == get_df_col(wc, 
           p_df, 
           'pseudochrom',
           allow_multiple=True)

False

array(['CLU-h2kbKI_HO', 'hTREM2_KI_HET', 'hTREM2_KI_WT',
       'hgenotype1_genotype2'], dtype=object)

In [8]:
# get all the pseudo chromosomes that are needed 
# need for the given genotype
wc = {'genotype': ('hgenotype1_genotype2')}
eep = get_cfg_entries(wc, 
     p_df,
     config['ref']['pseudochrom']['fa'])

In [6]:
eep

['ref/pseudochrom/hgenotype1/ref.fa', 'ref/pseudochrom/hgenotype2/ref.fa']

In [7]:
# output
get_cfg_entries(wc, 
                df,
                config['ref']['pseudochrom']['fa_merge'])

['ref/genome/hgenotype1_genotype2/merged.fa']

## 231102 adding a pseudochrom metadata table

In [3]:
# config_tsv = '231020_config.tsv'
config_tsv = 'test_pseudochrom_parsing_config.tsv'
meta_tsv = 'mouse_metadata.tsv'
auto_dedupe = True

df = parse_config_file(config_tsv,
                       meta_tsv,
                       auto_dedupe=auto_dedupe)

In [4]:
df.head()

,batch,platform,fname,basename,path,flowcell,file_stem,chop_num,sample_temp,mouse_id,...,tissue,pseudochrom_needed,pseudochrom,human_gene,mouse_gene,sample,biorep_num,dataset,source,cerberus_run
0,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_12670_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_12670_lig-blk_1,1.0,ad003_12670,12670,...,HC,True,"(hCLU,)",CLU,Clu,CLU-h2kbKI_HO_F_4_months_HC,1,CLU-h2kbKI_HO_F_4_months_HC_1,ad003_CLU-h2kbKI_HO_F_4_months_HC,1
1,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_12670_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_12670_lig-blk_2,1.0,ad003_12670,12670,...,HC,True,"(hCLU,)",CLU,Clu,CLU-h2kbKI_HO_F_4_months_HC,1,CLU-h2kbKI_HO_F_4_months_HC_1,ad003_CLU-h2kbKI_HO_F_4_months_HC,1
2,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad004_18634_lig-blk_1_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad004_18634_lig-blk_1,1.0,ad004_18634,18634,...,HC,True,"(hTREM2,)",TREM2,NaN,hTREM2_KI_HET_M_8_weeks_HC,1,hTREM2_KI_HET_M_8_weeks_HC_1,ad004_hTREM2_KI_HET_M_8_weeks_HC,2
3,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad004_18503_lig-blk_1_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad004_18503_lig-blk_1,1.0,ad004_18503,18503,...,HC,True,"(hTREM2,)",TREM2,NaN,hTREM2_KI_WT_M_8_weeks_HC,1,hTREM2_KI_WT_M_8_weeks_HC_1,ad004_hTREM2_KI_WT_M_8_weeks_HC,3
4,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad004_18503_lig-blk_2_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad004_18503_lig-blk_2,1.0,ad004_18503,18503,...,HC,True,"(hTREM2,)",TREM2,NaN,hTREM2_KI_WT_M_8_weeks_HC,1,hTREM2_KI_WT_M_8_weeks_HC_1,ad004_hTREM2_KI_WT_M_8_weeks_HC,3


In [27]:
p_meta_file = 'pseudochromosome_metadata.tsv'

In [28]:
# get a table that matches genotype + pseudochrom + human gene + mouse gene
temp = df[['genotype', 'pseudochrom']].explode('pseudochrom')
p_meta = pd.read_csv(p_meta_file, sep='\t')
p_meta.fillna('dummy', inplace=True)
p_meta = temp.merge(p_meta, on='pseudochrom')

In [29]:
p_meta

,genotype,pseudochrom,human_gene,mouse_gene,locus_type,notes
0,hTREM2_KI_HET,hTREM2,TREM2,dummy,chimeric,dummy
1,hTREM2_KI_WT,hTREM2,TREM2,dummy,chimeric,dummy
2,hTREM2_KI_WT,hTREM2,TREM2,dummy,chimeric,dummy
3,hgenotype1_genotype2,hgenotype1,dummy,dummy,dummy,dummy
4,hgenotype1_genotype2,hgenotype2,dummy,dummy,dummy,dummy


## 231102 debug

In [ ]:
# config_tsv = '231020_config.tsv'
config_tsv = 'test_pseudochrom_parsing_config.tsv'
meta_tsv = 'mouse_metadata.tsv'
auto_dedupe = True

df = parse_config_file(config_tsv,
                       meta_tsv,
                       auto_dedupe=auto_dedupe)

> /Users/fairliereese/Documents/programming/mortazavi_lab/bin/modelad_pipeline/proc_update/sm_utils.py(164)parse_config_file()
    162     # but first sort on study and sample such that they will always be ordered in the same way
    163     # this should freeze our results
--> 164     gb_cols = ['study', 'genotype', 'sex', 'age', 'tissue']
    165     df = df.sort_values(by=gb_cols, ascending=True)
    166     temp = df.copy(deep=True)



ipdb>  df.head()


    batch platform                                              fname  \
0  230516      ONT  /share/crsp/lab/seyedam/share/ad_nanopore_tmp/...   
1  230516      ONT  /share/crsp/lab/seyedam/share/ad_nanopore_tmp/...   
2  230516      ONT  /share/crsp/lab/seyedam/share/ad_nanopore_tmp/...   
3  230516      ONT  /share/crsp/lab/seyedam/share/ad_nanopore_tmp/...   
4  230516      ONT  /share/crsp/lab/seyedam/share/ad_nanopore_tmp/...   

                            basename  \
0     ad003_12670_lig-blk_1_t1.fastq   
1     ad003_12670_lig-blk_2_t1.fastq   
2  ad004_18503_lig-blk_1_t1.fastq.gz   
3  ad004_18503_lig-blk_2_t1.fastq.gz   
4  ad004_18634_lig-blk_1_t1.fastq.gz   

                                                path  flowcell  \
0  /share/crsp/lab/seyedam/share/ad_nanopore_tmp/...         1   
1  /share/crsp/lab/seyedam/share/ad_nanopore_tmp/...         2   
2  /share/crsp/lab/seyedam/share/ad_nanopore_tmp/...         1   
3  /share/crsp/lab/seyedam/share/ad_nanopore_tmp/...    

ipdb>  df.loc[df.genotype=='hgenotype1_genotype2']


    batch platform                                  fname  \
5  230516      ONT  /test/ad004_666_lig-blk_1_t1.fastq.gz   

                          basename   path  flowcell            file_stem  \
5  ad004_666_lig-blk_1_t1.fastq.gz  /test         1  ad004_666_lig-blk_1   

   chop_num sample_temp  mouse_id  ...      age tissue pseudochrom_needed  \
5       1.0   ad004_666       666  ...  8_weeks     HC               True   

                pseudochrom        human_gene        mouse_gene  \
5  (hgenotype1, hgenotype2)  (hgene1, hgene2)  (hgene1, mgene2)   

                              sample biorep_num  \
5  hgenotype1_genotype2_M_8_weeks_HC          1   

                               dataset  \
5  hgenotype1_genotype2_M_8_weeks_HC_1   

                                    source  
5  ad004_hgenotype1_genotype2_M_8_weeks_HC  

[1 rows x 23 columns]


ipdb>  q


## 231102 deubg

In [3]:
# config_tsv = '231020_config.tsv'
config_tsv = '231020_config.tsv'
# config_tsv = 'test_pseudochrom_parsing_config.tsv'
meta_tsv = 'mouse_metadata.tsv'
auto_dedupe = True

df = parse_config_file(config_tsv,
                       meta_tsv,
                       auto_dedupe=auto_dedupe,
                       include_pseudochrom=True)

/Users/fairliereese/Documents/programming/mortazavi_lab/bin/modelad_pipeline/proc_update/sm_utils.py:126: UserWarning: Config found non-pseudochrom mouse w/ genotypes ['hABKI_SWE'], is this expected?
  warnings.warn(f'Config found non-pseudochrom mouse w/ genotypes {genotypes}, is this expected?')


In [4]:
df.head()

,batch,platform,fname,basename,path,flowcell,file_stem,chop_num,sample_temp,mouse_id,...,tissue,pseudochrom_needed,pseudochrom,human_gene,mouse_gene,sample,biorep_num,dataset,source,cerberus_run
0,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10725_lig-blk_2_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad002_10725_lig-blk_2,1.0,ad002_10725,10725,...,CTX,False,"(dummy,)","(dummy,)","(dummy,)",5xFADHEMI_M_18_months_CTX,1,5xFADHEMI_M_18_months_CTX_1,ad002_5xFADHEMI_M_18_months_CTX,1
1,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10725_lig-ss_blk_1_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad002_10725_lig-ss_blk_1,1.0,ad002_10725,10725,...,CTX,False,"(dummy,)","(dummy,)","(dummy,)",5xFADHEMI_M_18_months_CTX,1,5xFADHEMI_M_18_months_CTX_1,ad002_5xFADHEMI_M_18_months_CTX,1
2,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10721_lig-blk_2_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad002_10721_lig-blk_2,1.0,ad002_10721,10721,...,CTX,False,"(dummy,)","(dummy,)","(dummy,)",5xFADWT_M_18_months_CTX,1,5xFADWT_M_18_months_CTX_1,ad002_5xFADWT_M_18_months_CTX,2
3,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10721_lig-ss_blk_1_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad002_10721_lig-ss_blk_1,1.0,ad002_10721,10721,...,CTX,False,"(dummy,)","(dummy,)","(dummy,)",5xFADWT_M_18_months_CTX,1,5xFADWT_M_18_months_CTX_1,ad002_5xFADWT_M_18_months_CTX,2
4,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_12517_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_12517_lig-blk_1,1.0,ad003_12517,12517,...,HC,True,"(hCLU,)","(CLU,)","(Clu,)",5xCLU-h2kbKI_HO_F_4_months_HC,1,5xCLU-h2kbKI_HO_F_4_months_HC_1,ad003_5xCLU-h2kbKI_HO_F_4_months_HC,3


In [5]:
df.genotype.unique()

array(['5xFADHEMI', '5xFADWT', '5xCLU-h2kbKI_HO', 'CLU-h2kbKI_HO',
       'hTREM2-R47HKI_HET', 'hTREM2_KI_HET', 'hTREM2_KI_WT', '5xBIN1_HO',
       'BIN1_HO', 'C57B6J', 'hABKI_SWE', 'ABCA7_HO'], dtype=object)

In [6]:
df.loc[df.genotype=='5xFADHEMI']

,batch,platform,fname,basename,path,flowcell,file_stem,chop_num,sample_temp,mouse_id,...,tissue,pseudochrom_needed,pseudochrom,human_gene,mouse_gene,sample,biorep_num,dataset,source,cerberus_run
0,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10725_lig-blk_2_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad002_10725_lig-blk_2,1.0,ad002_10725,10725,...,CTX,False,"(dummy,)","(dummy,)","(dummy,)",5xFADHEMI_M_18_months_CTX,1,5xFADHEMI_M_18_months_CTX_1,ad002_5xFADHEMI_M_18_months_CTX,1
1,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10725_lig-ss_blk_1_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad002_10725_lig-ss_blk_1,1.0,ad002_10725,10725,...,CTX,False,"(dummy,)","(dummy,)","(dummy,)",5xFADHEMI_M_18_months_CTX,1,5xFADHEMI_M_18_months_CTX_1,ad002_5xFADHEMI_M_18_months_CTX,1
10,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11616_lig-blk_1,1.0,ad003_11616,11616,...,HC,False,"(dummy,)","(dummy,)","(dummy,)",5xFADHEMI_F_4_months_HC,1,5xFADHEMI_F_4_months_HC_1,ad003_5xFADHEMI_F_4_months_HC,4
11,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_11616_lig-blk_2,1.0,ad003_11616,11616,...,HC,False,"(dummy,)","(dummy,)","(dummy,)",5xFADHEMI_F_4_months_HC,1,5xFADHEMI_F_4_months_HC_1,ad003_5xFADHEMI_F_4_months_HC,4
12,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11617_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11617_lig-blk_1,1.0,ad003_11617,11617,...,HC,False,"(dummy,)","(dummy,)","(dummy,)",5xFADHEMI_F_4_months_HC,2,5xFADHEMI_F_4_months_HC_2,ad003_5xFADHEMI_F_4_months_HC,4
13,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11617_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_11617_lig-blk_2,1.0,ad003_11617,11617,...,HC,False,"(dummy,)","(dummy,)","(dummy,)",5xFADHEMI_F_4_months_HC,2,5xFADHEMI_F_4_months_HC_2,ad003_5xFADHEMI_F_4_months_HC,4
14,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11625_lig-blk_1_t2.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11625_lig-blk_1,2.0,ad003_11625,11625,...,HC,False,"(dummy,)","(dummy,)","(dummy,)",5xFADHEMI_F_4_months_HC,3,5xFADHEMI_F_4_months_HC_3,ad003_5xFADHEMI_F_4_months_HC,4
15,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11625_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_11625_lig-blk_2,1.0,ad003_11625,11625,...,HC,False,"(dummy,)","(dummy,)","(dummy,)",5xFADHEMI_F_4_months_HC,3,5xFADHEMI_F_4_months_HC_3,ad003_5xFADHEMI_F_4_months_HC,4


In [7]:
#   study=ad002

#   genotype=5xFADHEMI
#   sex=M
#   age=18_months
#   tissue=CTX
wc = {'study': 'ad002', 'genotype': '5xFADHEMI',
      'sex': 'M', 'age': '18_months', 'tissue': 'CTX'}

In [8]:
get_cfg_entries(wc, df, config['merge']['sort_bam'])

['data/merge/ad002/5xFADHEMI_M_18_months_CTX_1_sorted.bam']

## 231102 adding hgene / mgene parsing to config parser

In [12]:
# config_tsv = '231020_config.tsv'
config_tsv = 'test_pseudochrom_parsing_config.tsv'
meta_tsv = 'mouse_metadata.tsv'
auto_dedupe = True

df = parse_config_file(config_tsv,
                       meta_tsv,
                       auto_dedupe=auto_dedupe,
                       include_pseudochrom=True)

In [13]:
df.loc[df.genotype=='hgenotype1_genotype2']

,batch,platform,fname,basename,path,flowcell,file_stem,chop_num,sample_temp,mouse_id,...,tissue,pseudochrom_needed,pseudochrom,human_gene,mouse_gene,sample,biorep_num,dataset,source,cerberus_run
5,230516,ONT,/test/ad004_666_lig-blk_1_t1.fastq.gz,ad004_666_lig-blk_1_t1.fastq.gz,/test,1,ad004_666_lig-blk_1,1.0,ad004_666,666,...,HC,True,"(hgenotype1, hgenotype2)","(hgene1, hgene2)","(hgene1, mgene2)",hgenotype1_genotype2_M_8_weeks_HC,1,hgenotype1_genotype2_M_8_weeks_HC_1,ad004_hgenotype1_genotype2_M_8_weeks_HC,4


In [14]:
# df['pseudochromosome_names'] = df.apply(lambda x: tuple(sorted(x.pseudochromosome_names)), axis=1)

In [15]:
# df.loc[df.genotype=='hgenotype1_genotype2']

In [16]:
wc = {'pseudochrom': ('hgenotype1', 'hgenotype2')}
# get_df_col(wc, df, 'genotype')
# get_cfg_entries(wc, 
#                 df,
#                 config['ref']['pseudochrom']['fa_merge'])

In [17]:
# get all the pseudo chromosomes that are needed 
# need for the given genotype
get_cfg_entries(wc, 
     df,
     config['ref']['pseudochrom']['fa'])

['ref/pseudochrom/hgenotype1/ref.fa', 'ref/pseudochrom/hgenotype2/ref.fa']

In [18]:
# output
get_cfg_entries(wc, 
                df,
                config['ref']['pseudochrom']['fa_merge'])

['ref/genome/hgenotype1_genotype2/merged.fa']

## 231023 -- get the cerberus run = 1 for the first agg runs

In [3]:
config_tsv = '231020_config.tsv'

meta_tsv = 'mouse_metadata.tsv'
# meta_tsv = 'mouse_test_metadata.tsv'
auto_dedupe = True

df = parse_config_file(config_tsv,
                       meta_tsv,
                       auto_dedupe=auto_dedupe,
                       include_humanized=False)


In [4]:
df.head()

,batch,platform,fname,basename,path,flowcell,file_stem,chop_num,sample_temp,mouse_id,...,sex,age,tissue,humanized,humanized_loci,sample,biorep_num,dataset,source,cerberus_run
0,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10725_lig-blk_2_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad002_10725_lig-blk_2,1.0,ad002_10725,10725,...,M,18_months,CTX,False,NaN,5xFADHEMI_M_18_months_CTX,1,5xFADHEMI_M_18_months_CTX_1,ad002_5xFADHEMI_M_18_months_CTX,1
1,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10725_lig-ss_blk_1_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad002_10725_lig-ss_blk_1,1.0,ad002_10725,10725,...,M,18_months,CTX,False,NaN,5xFADHEMI_M_18_months_CTX,1,5xFADHEMI_M_18_months_CTX_1,ad002_5xFADHEMI_M_18_months_CTX,1
2,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10721_lig-blk_2_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad002_10721_lig-blk_2,1.0,ad002_10721,10721,...,M,18_months,CTX,False,NaN,5xFADWT_M_18_months_CTX,1,5xFADWT_M_18_months_CTX_1,ad002_5xFADWT_M_18_months_CTX,2
3,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10721_lig-ss_blk_1_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad002_10721_lig-ss_blk_1,1.0,ad002_10721,10721,...,M,18_months,CTX,False,NaN,5xFADWT_M_18_months_CTX,1,5xFADWT_M_18_months_CTX_1,ad002_5xFADWT_M_18_months_CTX,2
4,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11616_lig-blk_1,1.0,ad003_11616,11616,...,F,4_months,HC,False,NaN,5xFADHEMI_F_4_months_HC,1,5xFADHEMI_F_4_months_HC_1,ad003_5xFADHEMI_F_4_months_HC,3


In [5]:
cfg_entry = config['cerberus']['ics']
wc = {}
get_first_cerb_entry(wc, df, cfg_entry)

'data/cerberus/ad002/5xFADHEMI_M_18_months_CTX_1_ic.tsv'

In [6]:
cfg_entry = config['cerberus']['ends']
wc = {'end_mode': 'tss'}
get_first_cerb_entry(wc, df, cfg_entry)

'data/cerberus/ad002/5xFADHEMI_M_18_months_CTX_1_tss.bed'

## 231022 -- some other suff

In [19]:
wc = {'study': 'ad005',
      'genotype': '5xBIN1_HO',
      'sex': 'F',
      'age': '4_months',
      'tissue': 'HC'}

config_tsv = '231020_config.tsv'
# config_tsv = 'test_local_config.tsv'
meta_tsv = 'mouse_metadata.tsv'
# meta_tsv = 'mouse_test_metadata.tsv'
auto_dedupe = True

df = parse_config_file(config_tsv,
                       meta_tsv,
                       auto_dedupe=auto_dedupe,
                       include_humanized=False)
get_cfg_entries(wc, df, config['merge']['sort_bam'])

['data/merge/ad005/5xBIN1_HO_F_4_months_HC_3_sorted.bam',
 'data/merge/ad005/5xBIN1_HO_F_4_months_HC_2_sorted.bam',
 'data/merge/ad005/5xBIN1_HO_F_4_months_HC_1_sorted.bam']

## 231020 pipeline restructuring -- get bws for each mouse

In [11]:
wc = {'study': 'ad003',
      'genotype': '5xFADHEMI',
      'sex': 'F',
      'age': '4_months',
      'tissue': 'HC'}

config_tsv = '231020_config.tsv'
# config_tsv = 'test_local_config.tsv'
meta_tsv = 'mouse_metadata.tsv'
# meta_tsv = 'mouse_test_metadata.tsv'
auto_dedupe = True

df = parse_config_file(config_tsv,
                       meta_tsv,
                       auto_dedupe=auto_dedupe,
                       include_humanized=False)

In [15]:
# expand(get_cfg_entries(wc, df,
#                         config['merge']['bw']),
#        strand=['fwd', 'rev'])

In [18]:
expand(expand(config['merge']['bw'],
        zip,
        study=df.study.tolist(),
        genotype=df.genotype.tolist(),
        sex=df.sex.tolist(),
        age=df.age.tolist(),
        tissue=df.tissue.tolist(),
        biorep_num=df.biorep_num.tolist(),
        allow_missing=True),
        strand=['fwd', 'rev'])

['data/merge/ad002/5xFADHEMI_M_18_months_CTX_1_fwd.bw',
 'data/merge/ad002/5xFADHEMI_M_18_months_CTX_1_rev.bw',
 'data/merge/ad002/5xFADHEMI_M_18_months_CTX_1_fwd.bw',
 'data/merge/ad002/5xFADHEMI_M_18_months_CTX_1_rev.bw',
 'data/merge/ad002/5xFADWT_M_18_months_CTX_1_fwd.bw',
 'data/merge/ad002/5xFADWT_M_18_months_CTX_1_rev.bw',
 'data/merge/ad002/5xFADWT_M_18_months_CTX_1_fwd.bw',
 'data/merge/ad002/5xFADWT_M_18_months_CTX_1_rev.bw',
 'data/merge/ad003/5xFADHEMI_F_4_months_HC_1_fwd.bw',
 'data/merge/ad003/5xFADHEMI_F_4_months_HC_1_rev.bw',
 'data/merge/ad003/5xFADHEMI_F_4_months_HC_1_fwd.bw',
 'data/merge/ad003/5xFADHEMI_F_4_months_HC_1_rev.bw',
 'data/merge/ad003/5xFADHEMI_F_4_months_HC_2_fwd.bw',
 'data/merge/ad003/5xFADHEMI_F_4_months_HC_2_rev.bw',
 'data/merge/ad003/5xFADHEMI_F_4_months_HC_2_fwd.bw',
 'data/merge/ad003/5xFADHEMI_F_4_months_HC_2_rev.bw',
 'data/merge/ad003/5xFADHEMI_F_4_months_HC_3_fwd.bw',
 'data/merge/ad003/5xFADHEMI_F_4_months_HC_3_rev.bw',
 'data/merge/ad003/5

## 231020 pipeline restructuring -- sanitizing humanized input

In [3]:
wc = {'study': 'ad003',
      'genotype': '5xFADHEMI',
      'sex': 'F',
      'age': '4_months',
      'tissue': 'HC',
      'cerberus_run': '2'}

config_tsv = '231020_config.tsv'
# config_tsv = 'test_local_config.tsv'
meta_tsv = 'mouse_metadata.tsv'
# meta_tsv = 'mouse_test_metadata.tsv'
auto_dedupe = True

df = parse_config_file(config_tsv,
                       meta_tsv,
                       auto_dedupe=auto_dedupe,
                       include_humanized=False)

In [4]:
df[['study', 'genotype', 'sex', 'age', 'tissue', 'humanized']].drop_duplicates()

,study,genotype,sex,age,tissue,humanized
0,ad002,5xFADHEMI,M,18_months,CTX,False
2,ad002,5xFADWT,M,18_months,CTX,False
4,ad003,5xFADHEMI,F,4_months,HC,False
10,ad003,5xFADWT,F,4_months,HC,False
16,ad005,5xBIN1_HO,F,4_months,HC,False
22,ad005,BIN1_HO,F,4_months,HC,False
29,ad006,C57B6J,F,8_weeks,HC,False
35,ad006,C57B6J,M,8_weeks,HC,False
41,ad007,ABCA7_HO,F,4_months,HC,False


## 231020 pipeline restructuring -- more cerberus shenannigans

In [81]:
wc = {'study': 'ad003',
      'genotype': '5xFADHEMI',
      'sex': 'F',
      'age': '4_months',
      'tissue': 'HC',
      'cerberus_run': 2}
wc = {'study': 'ad003',
      'genotype': '5xFADHEMI',
      'sex': 'F',
      'age': '4_months',
      'tissue': 'HC'}

# config_tsv = '231018_config.tsv'
config_tsv = 'test_local_config.tsv'
meta_tsv = 'mouse_metadata.tsv'
auto_dedupe = True

df = parse_config_file(config_tsv,
                       meta_tsv,
                       auto_dedupe=auto_dedupe)

In [83]:
get_cfg_entries({'cerberus_run':df.cerberus_run.unique().tolist()},
                        df,
                        config['cerberus']['ca'])

['data/cerberus/ad003/5xFADHEMI_F_4_months_HC_1.h5',
 'data/cerberus/ad003/5xFADWT_F_4_months_HC_2.h5',
 'data/cerberus/ad005/BIN1_HO_F_4_months_HC_3.h5']

In [74]:
# need to get tss / tes by adding end mode wcs
# expand(['cerberus']['agg']['ends'],
#                                end_mode='tss')[0]

In [76]:
expand(get_cfg_entries(wc,
                        df,
                        config['cerberus']['agg']['ends']),
       end_mode='tss')[0]

['data/cerberus/agg/ad003_5xFADHEMI_F_4_months_HC_1_tss.bed']

## 231019 pipeline restructuring -- cerberus getting previous cerberus run

In [50]:
wc = {'study': 'ad003',
      'genotype': '5xFADHEMI',
      'sex': 'F',
      'age': '4_months',
      'tissue': 'HC',
      'cerberus_run': '2'}

# config_tsv = '231018_config.tsv'
config_tsv = 'test_local_config.tsv'
meta_tsv = 'mouse_metadata.tsv'
auto_dedupe = True

df = parse_config_file(config_tsv,
                       meta_tsv,
                       auto_dedupe=auto_dedupe)

In [51]:
df.cerberus_run.max()

'3'

In [60]:
get_cfg_entries({'cerberus_run':df.cerberus_run.max()},
                        df,
                        config['cerberus']['agg']['ends'])

['data/cerberus/agg/ad005_BIN1_HO_F_4_months_HC_3_{end_mode}.bed']

In [57]:
def get_prev_cerb_entry(wc, df, cfg_entry):
    """
    Get the previous config entry run for Cerberus. Ensure that 
    only one file meets these criteria.
    """
    prev_wc = {'cerberus_run': str(int(wc['cerberus_run'])-1)}
    file = get_cfg_entries(prev_wc, df, cfg_entry)
    assert len(file) == 1
    file = file[0]
    
    # add in the end mode if we have it
    if 'end_mode' in wc.keys():
        file = expand(file, 
                      zip,
                      end_mode=wc['end_mode'])
        assert len(file) == 1
        file = file[0]
    
    return file

In [58]:
get_prev_cerb_entry(wc, df, config['cerberus']['agg']['ics'])

'data/cerberus/agg/ad003_5xFADHEMI_F_4_months_HC_1_ic.tsv'

In [59]:
wc = {'study': 'ad003',
      'genotype': '5xFADHEMI',
      'sex': 'F',
      'age': '4_months',
      'tissue': 'HC',
      'end_mode': 'tss',
      'cerberus_run': '2'}

# config_tsv = '231018_config.tsv'
config_tsv = 'test_local_config.tsv'
meta_tsv = 'mouse_metadata.tsv'
auto_dedupe = True

df = parse_config_file(config_tsv,
                       meta_tsv,
                       auto_dedupe=auto_dedupe)

get_prev_cerb_entry(wc, df, config['cerberus']['agg']['ends'])

'data/cerberus/agg/ad003_5xFADHEMI_F_4_months_HC_1_tss.bed'

In [8]:
prev_wc = {'cerberus_run': str(int(wc['cerberus_run'])-1)}
prev_wc

get_cfg_entries({'cerberus_run': str(int(wc['cerberus_run'])-1)}, df, config['cerberus']['ics'])

['data/cerberus/ad003/5xCLU-h2kbKI_HO_F_4_months_HC_1_ic.tsv']

In [28]:
get_cfg_entries({'cerberus_run':df.cerberus_run.max()},
                        df,
                        config['cerberus']['agg']['ics'])

['data/cerberus/agg/ad005_BIN1_HO_F_4_months_HC_3_ic.tsv']

## 231019 pipeline restructuring -- cerberus getting source from current wcs (should be the study_genotype_sex_age_tissue combo again)

In [6]:
wc = {'study': 'ad003',
      'genotype': '5xFADHEMI',
      'sex': 'F',
      'age': '4_months',
      'tissue': 'HC',
      'end_mode': 'tss',
      'cerberus_run': '2'}

config_tsv = '231018_config.tsv'
# config_tsv = 'test_local_config.tsv'
meta_tsv = 'mouse_metadata.tsv'
auto_dedupe = True

df = parse_config_file(config_tsv,
                       meta_tsv,
                       auto_dedupe=auto_dedupe)

In [7]:
get_df_col(wc, df, 'source')

'ad003_5xFADHEMI_F_4_months_HC'

## 231019 pipeline restructuring -- cerberus stuff

In [118]:
config_tsv = '231018_config.tsv'
# config_tsv = 'test_local_config.tsv'
meta_tsv = 'mouse_metadata.tsv'
auto_dedupe = True

df = parse_config_file(config_tsv,
                       meta_tsv,
                       auto_dedupe=auto_dedupe)


In [130]:
# assign a cerberus run to each "sample" (study+genotype+sex+age+tissue)
# but first sort on study and sample such that they will always be ordered in the same way
# this should freeze our results
gb_cols = ['study', 'genotype', 'sex', 'age', 'tissue']
df = df.sort_values(by=gb_cols, ascending=True)
temp = df.copy(deep=True)
temp = temp[gb_cols].groupby(gb_cols).count().reset_index()
temp['cerberus_run'] = [i+1 for i in range(len(temp.index))]
df = df.merge(temp, how='left', on=gb_cols)

In [132]:
df.head(10)

,batch,platform,fname,basename,path,flowcell,file_stem,chop_num,sample_temp,mouse_id,study,genotype,sex,age,tissue,sample,biorep_num,dataset,cerberus_run
0,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_12517_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_12517_lig-blk_1,1.0,ad003_12517,12517,ad003,5xCLU-h2kbKI_HO,F,4_months,HC,5xCLU-h2kbKI_HO_F_4_months_HC,1,5xCLU-h2kbKI_HO_F_4_months_HC_1,1
1,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_12517_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_12517_lig-blk_2,1.0,ad003_12517,12517,ad003,5xCLU-h2kbKI_HO,F,4_months,HC,5xCLU-h2kbKI_HO_F_4_months_HC,1,5xCLU-h2kbKI_HO_F_4_months_HC_1,1
2,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_12649_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_12649_lig-blk_1,1.0,ad003_12649,12649,ad003,5xCLU-h2kbKI_HO,F,4_months,HC,5xCLU-h2kbKI_HO_F_4_months_HC,2,5xCLU-h2kbKI_HO_F_4_months_HC_2,1
3,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_12649_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_12649_lig-blk_2,1.0,ad003_12649,12649,ad003,5xCLU-h2kbKI_HO,F,4_months,HC,5xCLU-h2kbKI_HO_F_4_months_HC,2,5xCLU-h2kbKI_HO_F_4_months_HC_2,1
4,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_12659_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_12659_lig-blk_1,1.0,ad003_12659,12659,ad003,5xCLU-h2kbKI_HO,F,4_months,HC,5xCLU-h2kbKI_HO_F_4_months_HC,3,5xCLU-h2kbKI_HO_F_4_months_HC_3,1
5,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_12659_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_12659_lig-blk_2,1.0,ad003_12659,12659,ad003,5xCLU-h2kbKI_HO,F,4_months,HC,5xCLU-h2kbKI_HO_F_4_months_HC,3,5xCLU-h2kbKI_HO_F_4_months_HC_3,1
6,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11616_lig-blk_1,1.0,ad003_11616,11616,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,1,5xFADHEMI_F_4_months_HC_1,2
7,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_11616_lig-blk_2,1.0,ad003_11616,11616,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,1,5xFADHEMI_F_4_months_HC_1,2
8,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11617_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11617_lig-blk_1,1.0,ad003_11617,11617,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,2,5xFADHEMI_F_4_months_HC_2,2
9,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11617_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_11617_lig-blk_2,1.0,ad003_11617,11617,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,2,5xFADHEMI_F_4_months_HC_2,2


## 231019 pipeline restructuring -- lapa filtering settings

In [105]:
t_novs = config['lapa']['filt']['t_novs']
g_novs = config['lapa']['filt']['g_novs']
filt_spikes = config['lapa']['filt']['remove_spikes']

In [109]:
type(filt_spikes)

bool

## 231019 pipeline restructuring -- lapa input

In [91]:
def subset_df_on_wcs(wc, df):
    """
    Return a copy of the input metadata df limited to the wildcards
    """
    temp = df.copy(deep=True)
    for key, item in wc.items():
        
        # for entries that we don't have in the df, such as LAPA end mode
        if key not in df.columns:
            continue
        
        # if we're given a list of possibilities
        if type(item) == list:
            temp = temp.loc[temp[key].isin(item)]
        else:
            temp = temp.loc[temp[key] == item]
    return temp

In [92]:
def get_cfg_entries(wc, df, cfg_entry, return_df=False):
    """
    Expand a config entry based on the wildcards and the
    values in df that satisfy these wildcards

    Parameters:
        return_df (bool): Return DataFrame with 'file' column
            as opposed to list of files. Default: False
    """
    temp = subset_df_on_wcs(wc, df)

    # cols = ['study', 'genotype', 'sex',
    #         'age', 'tissue', 'biorep_num',
    #         'flowcell']

    study = temp.study.tolist()
    genotype = temp.genotype.tolist()
    sex = temp.sex.tolist()
    age = temp.age.tolist()
    tissue = temp.tissue.tolist()
    biorep_num = temp.biorep_num.tolist()
    flowcell = temp.flowcell.tolist()

    files = expand(cfg_entry,
                   zip,
                   study=study,
                   genotype=genotype,
                   sex=sex,
                   age=age,
                   tissue=tissue,
                   biorep_num=biorep_num,
                   flowcell=flowcell,
                   allow_missing=True)

    temp['file'] = files

    # make sure we only take unique ones
    # cols.append('file')
    # temp = temp[cols]
    temp = temp.drop_duplicates(subset='file', keep='first')
    files = temp['file'].tolist()

    if return_df:
        return temp
    else:
        return files

In [101]:
def get_lapa_settings(wc, cfg_entry, kind):
    """
    Get the command name or file output name
    given whether we're running LAPA in tss or tes mode

    Parameters:
        lapa_ends (str): Config entry 
        kind (str): {'temp_file', 'lapa_cmd'}
    """
    
    # get the output file from the lapa run
    lapa_ends = get_cfg_entries(wc, df, cfg_entry, return_df=False)
    
    # now expand for end mode
    lapa_ends = expand(lapa_ends, 
                       zip=True,
                       end_mode=wc['end_mode'])    
    assert len(list(set(lapa_ends))) == 1
    lapa_ends = lapa_ends[0]
    
    if kind == 'temp_file':
        temp = os.path.dirname(lapa_ends)+'/'
        if wc['end_mode'] == 'tes':
            temp += 'polyA_clusters.bed'
        elif wc['end_mode'] == 'tss':
            temp += 'tss_clusters.bed'
        return temp
    elif kind == 'lapa_cmd':
        if wc['end_mode'] == 'tes':
            return 'lapa'
        elif wc['end_mode'] == 'tss':
            return 'lapa_tss'

In [102]:
# get all files from bioreps of same genotype, sex, age, tissue
wc = {'study': 'ad003',
      'genotype': '5xFADHEMI',
      'sex': 'F',
      'age': ['4_months', '10_days'],
      'tissue': 'HC',
      'end_mode': 'tss'}

cfg_entry = config['lapa']['ends']
get_lapa_settings(wc, cfg_entry, 'temp_file')

'data/lapa/ad003/tss/tss_clusters.bed'

In [103]:
get_lapa_settings(wc, cfg_entry, 'lapa_cmd')

'lapa_tss'

## 231019 pipeline restructuring -- talon input

In [71]:
# utilities related to defining snakemake rules
from snakemake.io import expand

def subset_df_on_wcs(wc, df):
    """
    Return a copy of the input metadata df limited to the wildcards
    """
    temp = df.copy(deep=True)
    for key, item in wc.items():
        if type(item) == list:
            temp = temp.loc[temp[key].isin(item)]
        else:
            temp = temp.loc[temp[key] == item]
    return temp

def get_df_col(wc, df, col):
    """
    From the metadata dataframe df, get the entries that satisfy
    the wildcards requirements and return the corresponding value
    from col. Ensure that this is always a 1:1 relationship, otherwise
    throw an error.
    """
    cols = [col] + [key for key, item in wc.items()]

    temp = subset_df_on_wcs(wc, df)
    temp = temp[cols].drop_duplicates()

    if len(temp.index) != 1:
        msg = 'Issues getting data from DF with wildcards'
        for key, item in wc.items():
            msg+=f'\n{key}: {item}'
        raise ValueError(msg)

    val = temp[col].tolist()[0]
    return val

def get_cfg_entries(wc, df, cfg_entry, return_df=False):
    """
    Expand a config entry based on the wildcards and the
    values in df that satisfy these wildcards
    
    Parameters:
        return_df (bool): Return DataFrame with 'file' column 
            as opposed to list of files. Default: False
    """
    temp = subset_df_on_wcs(wc, df)

    # cols = ['study', 'genotype', 'sex',
    #         'age', 'tissue', 'biorep_num',
    #         'flowcell']

    study = temp.study.tolist()
    genotype = temp.genotype.tolist()
    sex = temp.sex.tolist()
    age = temp.age.tolist()
    tissue = temp.tissue.tolist()
    biorep_num = temp.biorep_num.tolist()
    flowcell = temp.flowcell.tolist()

    files = expand(cfg_entry,
                   zip,
                   study=study,
                   genotype=genotype,
                   sex=sex,
                   age=age,
                   tissue=tissue,
                   biorep_num=biorep_num,
                   flowcell=flowcell)

    temp['file'] = files

    # make sure we only take unique ones
    # cols.append('file')
    # temp = temp[cols]
    temp = temp.drop_duplicates(subset='file', keep='first')
    files = temp['file'].tolist()
    
    if return_df:
        return temp
    else: 
        return files

In [110]:
# get all files from bioreps of same genotype, sex, age, tissue
wc = {'study': 'ad003',
      'genotype': '5xFADHEMI',
      'sex': 'F',
      'age': ['4_months', '10_days'],
      'tissue': 'HC'}

files = get_cfg_entries(wc, df, config['merge']['sort_bam'])
files
temp = get_cfg_entries(wc, df, cfg_entry, return_df=True)
temp

,batch,platform,fname,basename,path,flowcell,file_stem,chop_num,sample_temp,mouse_id,study,genotype,sex,age,tissue,sample,biorep_num,dataset,file
0,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11616_lig-blk_1,1.0,ad003_11616,11616,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,1,5xFADHEMI_F_4_months_HC_1,data/lapa/ad003/{end_mode}/5xFADHEMI_F_4_month...


In [112]:
temp = subset_df_on_wcs(wc, df)
temp


,batch,platform,fname,basename,path,flowcell,file_stem,chop_num,sample_temp,mouse_id,study,genotype,sex,age,tissue,sample,biorep_num,dataset
0,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11616_lig-blk_1,1.0,ad003_11616,11616,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,1,5xFADHEMI_F_4_months_HC_1
1,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_11616_lig-blk_2,1.0,ad003_11616,11616,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,1,5xFADHEMI_F_4_months_HC_1
2,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11617_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11617_lig-blk_1,1.0,ad003_11617,11617,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,2,5xFADHEMI_F_4_months_HC_2
3,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11617_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_11617_lig-blk_2,1.0,ad003_11617,11617,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,2,5xFADHEMI_F_4_months_HC_2
4,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11625_lig-blk_1_t2.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11625_lig-blk_1,2.0,ad003_11625,11625,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,3,5xFADHEMI_F_4_months_HC_3
5,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11625_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_11625_lig-blk_2,1.0,ad003_11625,11625,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,3,5xFADHEMI_F_4_months_HC_3


In [79]:
temp = df.loc[(df.study==wc['study'])&\
       (df.genotype==wc['genotype'])&\
       (df.sex==wc['sex'])&\
       (df.age.isin(wc['age']))&\
       (df.tissue==wc['tissue'])]
temp

,batch,platform,fname,basename,path,flowcell,file_stem,chop_num,sample_temp,mouse_id,study,genotype,sex,age,tissue,sample,biorep_num,dataset
0,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11616_lig-blk_1,1.0,ad003_11616,11616,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,1,5xFADHEMI_F_4_months_HC_1
1,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_11616_lig-blk_2,1.0,ad003_11616,11616,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,1,5xFADHEMI_F_4_months_HC_1
2,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11617_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11617_lig-blk_1,1.0,ad003_11617,11617,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,2,5xFADHEMI_F_4_months_HC_2
3,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11617_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_11617_lig-blk_2,1.0,ad003_11617,11617,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,2,5xFADHEMI_F_4_months_HC_2
4,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11625_lig-blk_1_t2.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11625_lig-blk_1,2.0,ad003_11625,11625,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,3,5xFADHEMI_F_4_months_HC_3
5,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11625_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_11625_lig-blk_2,1.0,ad003_11625,11625,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,3,5xFADHEMI_F_4_months_HC_3


In [74]:
temp[['sample', 'dataset', 'platform', 'file']]

,sample,dataset,platform,file
0,5xFADHEMI_F_4_months_HC,5xFADHEMI_F_4_months_HC_1,ONT,data/flowcell_merge/5xFADHEMI_F_4_months_HC_1.bam
2,5xFADHEMI_F_4_months_HC,5xFADHEMI_F_4_months_HC_2,ONT,data/flowcell_merge/5xFADHEMI_F_4_months_HC_2.bam
4,5xFADHEMI_F_4_months_HC,5xFADHEMI_F_4_months_HC_3,ONT,data/flowcell_merge/5xFADHEMI_F_4_months_HC_3.bam


In [75]:
temp

,batch,platform,fname,basename,path,flowcell,file_stem,chop_num,sample_temp,mouse_id,study,genotype,sex,age,tissue,sample,biorep_num,dataset,file
0,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11616_lig-blk_1,1.0,ad003_11616,11616,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,1,5xFADHEMI_F_4_months_HC_1,data/flowcell_merge/5xFADHEMI_F_4_months_HC_1.bam
2,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11617_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11617_lig-blk_1,1.0,ad003_11617,11617,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,2,5xFADHEMI_F_4_months_HC_2,data/flowcell_merge/5xFADHEMI_F_4_months_HC_2.bam
4,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11625_lig-blk_1_t2.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11625_lig-blk_1,2.0,ad003_11625,11625,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,3,5xFADHEMI_F_4_months_HC_3,data/flowcell_merge/5xFADHEMI_F_4_months_HC_3.bam


In [117]:
df.loc[df.study=='ad005']

,batch,platform,fname,basename,path,flowcell,file_stem,chop_num,sample_temp,mouse_id,study,genotype,sex,age,tissue,sample,biorep_num,dataset
24,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad005_13051_lig-blk_2_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad005_13051_lig-blk_2,1.0,ad005_13051,13051,ad005,BIN1_HO,F,4_months,HC,BIN1_HO_F_4_months_HC,3,BIN1_HO_F_4_months_HC_3
25,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad005_13036_lig-blk_2_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad005_13036_lig-blk_2,1.0,ad005_13036,13036,ad005,BIN1_HO,F,4_months,HC,BIN1_HO_F_4_months_HC,1,BIN1_HO_F_4_months_HC_1
26,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad005_13047_lig-blk_3_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad005_13047_lig-blk_3,1.0,ad005_13047,13047,ad005,BIN1_HO,F,4_months,HC,BIN1_HO_F_4_months_HC,2,BIN1_HO_F_4_months_HC_2
27,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad005_13051_lig-blk_1_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad005_13051_lig-blk_1,1.0,ad005_13051,13051,ad005,BIN1_HO,F,4_months,HC,BIN1_HO_F_4_months_HC,3,BIN1_HO_F_4_months_HC_3
28,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad005_13053_lig-blk_1_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad005_13053_lig-blk_1,1.0,ad005_13053,13053,ad005,5xBIN1_HO,F,4_months,HC,5xBIN1_HO_F_4_months_HC,3,5xBIN1_HO_F_4_months_HC_3
29,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad005_13022_lig-blk_2_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad005_13022_lig-blk_2,1.0,ad005_13022,13022,ad005,5xBIN1_HO,F,4_months,HC,5xBIN1_HO_F_4_months_HC,2,5xBIN1_HO_F_4_months_HC_2
30,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad005_13053_lig-blk_2_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad005_13053_lig-blk_2,1.0,ad005_13053,13053,ad005,5xBIN1_HO,F,4_months,HC,5xBIN1_HO_F_4_months_HC,3,5xBIN1_HO_F_4_months_HC_3
31,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad005_13019_lig-blk_1_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad005_13019_lig-blk_1,1.0,ad005_13019,13019,ad005,5xBIN1_HO,F,4_months,HC,5xBIN1_HO_F_4_months_HC,1,5xBIN1_HO_F_4_months_HC_1
32,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad005_13047_lig-blk_2_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad005_13047_lig-blk_2,1.0,ad005_13047,13047,ad005,BIN1_HO,F,4_months,HC,BIN1_HO_F_4_months_HC,2,BIN1_HO_F_4_months_HC_2
33,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad005_13022_lig-blk_1_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad005_13022_lig-blk_1,1.0,ad005_13022,13022,ad005,5xBIN1_HO,F,4_months,HC,5xBIN1_HO_F_4_months_HC,2,5xBIN1_HO_F_4_months_HC_2


## 231017 pipeline restructuring

In [4]:
configfile: 'config.yml'
config_tsv = '231018_config.tsv'
meta_tsv = 'mouse_metadata.tsv'
auto_dedupe = True
df = parse_config_file(config_tsv,
                       meta_tsv,
                       auto_dedupe=auto_dedupe)


In [13]:
wc = {'genotype': '5xFADHEMI',
      'sex': 'F',
      'age': ['4_months', '10_days'],
      'tissue': 'HC',
      'biorep_num': '1',
      'flowcell': '1'}

In [41]:
cfg_entry = config['talon_label']['sort_bam']
wc = {'genotype': '5xFADHEMI',
      'sex': 'F',
      'age': ['4_months', '10_days'],
      'tissue': 'HC',
      'biorep_num': '1'}

temp = subset_df_on_wcs(wc, df)
study = temp.study.tolist()
genotype = temp.genotype.tolist()
sex = temp.sex.tolist()
age = temp.age.tolist()
tissue = temp.tissue.tolist()
biorep_num = temp.biorep_num.tolist()
flowcell = temp.flowcell.tolist()

files = expand(cfg_entry,
               zip,
               study=study,
               genotype=genotype,
               sex=sex,
               age=age,
               tissue=tissue,
               biorep_num=biorep_num,
               flowcell=flowcell)
return files

['data/talon_label/5xFADHEMI_F_4_months_HC_1_1_labeled_sorted.bam',
 'data/talon_label/5xFADHEMI_F_4_months_HC_1_2_labeled_sorted.bam']

In [40]:
df.head()

,batch,platform,fname,basename,path,flowcell,file_stem,chop_num,sample_temp,mouse_id,study,genotype,sex,age,tissue,sample,biorep_num,dataset
0,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11616_lig-blk_1,1.0,ad003_11616,11616,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,1,5xFADHEMI_F_4_months_HC_1
1,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_11616_lig-blk_2,1.0,ad003_11616,11616,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,1,5xFADHEMI_F_4_months_HC_1
2,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11617_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11617_lig-blk_1,1.0,ad003_11617,11617,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,2,5xFADHEMI_F_4_months_HC_2
3,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11617_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_11617_lig-blk_2,1.0,ad003_11617,11617,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,2,5xFADHEMI_F_4_months_HC_2
4,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11625_lig-blk_1_t2.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11625_lig-blk_1,2.0,ad003_11625,11625,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,3,5xFADHEMI_F_4_months_HC_3


In [30]:
get_df_col(wc, df, 'fname')

'/share/crsp/lab/seyedam/share/ad_nanopore_tmp/AD003/ad003_trimfastq/ad003_11616_lig-blk_1_t1.fastq'

In [25]:
# replicates in modelad consist of 
# * genotype
# * sex
# * age
# * tissue